In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import glob

def hair_remove(image):
    # Normalize the image
    normalized_image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    
    # Convert to RGB (handles RGBA to RGB conversion)
    if normalized_image.shape[2] == 4:  # Check if it has an alpha channel
        normalized_image = cv2.cvtColor(normalized_image, cv2.COLOR_RGBA2RGB)
    
    # Convert to grayscale
    grayScale = cv2.cvtColor(normalized_image, cv2.COLOR_RGB2GRAY)
    
    # Create structuring element
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9)) 
    
    # Apply blackhat morphological operation
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    
    # Gaussian blur the blackhat image
    bhg = cv2.GaussianBlur(blackhat, (3, 3), cv2.BORDER_DEFAULT)
    
    # Thresholding to create a binary mask
    ret, mask = cv2.threshold(bhg, 10, 255, cv2.THRESH_BINARY)
    
    # Inpaint the normalized image using the mask
    dst = cv2.inpaint(normalized_image, mask.astype(np.uint8), 6, cv2.INPAINT_TELEA)  
    
    return dst

# Get current working directory
current_directory = os.getcwd()

# Create an output directory if it doesn't exist
output_directory = os.path.join(current_directory, "processed_images")
os.makedirs(output_directory, exist_ok=True)

for root, dirs, files in os.walk(current_directory):
    for dir in dirs:
        full_dir_path = os.path.join(root, dir)
        image_files = glob.glob(full_dir_path + '/*.png')
        
        for image_file in image_files:
            print(f"Processing image: {image_file}")  # Print the full path of the image
            image = np.asarray(Image.open(image_file))
            
            # Resize the image to 224x224 before processing
            resized_image = cv2.resize(image, (1024, 124), interpolation=cv2.INTER_LINEAR)

            # Process the resized image to remove hair
            processed_img = hair_remove(resized_image)

            # Save the processed image in the output directory
            output_file_path = os.path.join(output_directory, os.path.basename(image_file))
            processed_img_pil = Image.fromarray(processed_img)
            processed_img_pil.save(output_file_path)

print("Processing complete. Processed images saved in the 'processed_images' directory.")


Processing image: /home/anonymous/Documents/Projects/Skin-Cancer-Multimodal-Classification/PAD-UFES-20/images/images/PAT_290_445_988.png
Processing image: /home/anonymous/Documents/Projects/Skin-Cancer-Multimodal-Classification/PAD-UFES-20/images/images/PAT_1176_631_36.png
Processing image: /home/anonymous/Documents/Projects/Skin-Cancer-Multimodal-Classification/PAD-UFES-20/images/images/PAT_1202_722_827.png
Processing image: /home/anonymous/Documents/Projects/Skin-Cancer-Multimodal-Classification/PAD-UFES-20/images/images/PAT_787_1482_328.png
Processing image: /home/anonymous/Documents/Projects/Skin-Cancer-Multimodal-Classification/PAD-UFES-20/images/images/PAT_377_1580_534.png
Processing image: /home/anonymous/Documents/Projects/Skin-Cancer-Multimodal-Classification/PAD-UFES-20/images/images/PAT_1034_160_819.png
Processing image: /home/anonymous/Documents/Projects/Skin-Cancer-Multimodal-Classification/PAD-UFES-20/images/images/PAT_342_716_845.png
Processing image: /home/anonymous/Doc